In [9]:
from gurobipy import Model, GRB, quicksum

In [10]:
# Create a new model
m = Model("solar_farm")

In [11]:
# Number of sites
N = 20

In [12]:
# Decision variables
x = m.addVars(N, vtype=GRB.BINARY, name="x")  # Whether to build a farm at site n
s = m.addVars(N, vtype=GRB.INTEGER, name="s") # Number of small panels at site n
l = m.addVars(N, vtype=GRB.INTEGER, name="l") # Number of large panels at site n

In [13]:
# Objective: Minimize cost of setting up farms and installing panels
m.setObjective(quicksum(3e6 * x[n] + 5000 * s[n] + 25000 * l[n] for n in range(N)), GRB.MINIMIZE)

In [14]:
# Constraints

# Total production requirement
m.addConstr(quicksum(6000 * s[n] + 20000 * l[n] for n in range(N)) >= 10e6, "TotalProduction")

# Capacity constraints at each site
m.addConstrs((s[n] + 3 * l[n] <= 110 * x[n] for n in range(N)), "Capacity")

# Ratio of small to large panels at each site
m.addConstrs((s[n] >= 0.1 * l[n] for n in range(N)), "PanelRatio")

# Geographical and Deployment Constraints
m.addConstr(sum(x[n] for n in range(5)) == 3, "Sites1-5")
m.addConstr(sum(x[n] for n in range(5, 10)) <= 4, "Sites6-10")
m.addConstr(sum(x[n] for n in range(10, 15)) >= 2, "Sites11-15")
m.addConstr(x[15] + sum(x[n] for n in range(17, 20)) <= 1, "Exclusive16")
m.addConstr(x[16] + sum(x[n] for n in range(17, 20)) <= 1, "Exclusive17")

# Total small panels must exceed 30% of all panels
total_panels = quicksum(s[n] + l[n] for n in range(N))
total_small_panels = quicksum(s[n] for n in range(N))
m.addConstr(total_small_panels >= 0.3 * total_panels, "SmallPanelRequirement")

<gurobi.Constr *Awaiting Model Update*>

In [15]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 47 rows, 60 columns and 203 nonzeros
Model fingerprint: 0xd0255bd0
Variable types: 0 continuous, 60 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+04]
  Objective range  [5e+03, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve time: 0.00s
Presolved: 47 rows, 60 columns, 203 nonzeros
Variable types: 0 continuous, 60 integer (20 binary)

Root relaxation: objective 5.345224e+07, 71 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5.3452e+07    0    3          - 5.3452e+07      -     -    0s
H    0     0                    5.375000e+07 5.3452e+07  0.55%     -    0s
H    0

In [16]:
# Print the solution
if m.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for n in range(N):
        if x[n].X > 0.5:  # Site is chosen
            print(f"Site {n+1}: Build with {s[n].X} small panels and {l[n].X} large panels")
else:
    print("No optimal solution found.")

Optimal solution found:
Site 2: Build with 5.0 small panels and 35.0 large panels
Site 3: Build with 5.0 small panels and 35.0 large panels
Site 4: Build with 20.0 small panels and 30.0 large panels
Site 6: Build with 110.0 small panels and 0.0 large panels
Site 7: Build with 5.0 small panels and 35.0 large panels
Site 8: Build with 5.0 small panels and 35.0 large panels
Site 9: Build with 5.0 small panels and 35.0 large panels
Site 11: Build with 5.0 small panels and 35.0 large panels
Site 12: Build with 5.0 small panels and 35.0 large panels
Site 13: Build with 5.0 small panels and 35.0 large panels
Site 14: Build with 110.0 small panels and 0.0 large panels
Site 15: Build with 110.0 small panels and -0.0 large panels
Site 16: Build with 5.0 small panels and 35.0 large panels
Site 17: Build with 5.0 small panels and 35.0 large panels
